In [135]:
from pdf_extractor import *
import os
import glob

In [136]:
path = "./"
pdfs = glob.glob("{}/*.pdf".format(path))
pdfs

['./20200621.pdf',
 './20200823.pdf',
 './20201018.pdf',
 './20210523.pdf',
 './20201213.pdf',
 './20210509.pdf',
 './20201011.pdf',
 './20200816.pdf',
 './20200712.pdf',
 './20210315.pdf',
 './20201115.pdf',
 './20200920.pdf',
 './20201220.pdf',
 './20210103.pdf',
 './20210411.pdf',
 './20210307.pdf',
 './20210425.pdf',
 './20210418.pdf',
 './20210328.pdf',
 './20201129.pdf',
 './20210228.pdf',
 './20200809.pdf',
 './20201122.pdf',
 './20210207.pdf',
 './20200719.pdf',
 './20200927.pdf',
 './20210321.pdf',
 './20210405.pdf',
 './20200705.pdf',
 './20210614.pdf',
 './20201025.pdf',
 './20210620.pdf',
 './20210604.pdf',
 './20201206.pdf',
 './20201227.pdf',
 './20210516.pdf',
 './20210124.pdf',
 './20201101.pdf',
 './20200913.pdf',
 './20200906.pdf',
 './20200830.pdf',
 './20210221.pdf',
 './20210131.pdf',
 './20200802.pdf',
 './20200628.pdf',
 './20210110.pdf',
 './20210530.pdf']

In [74]:


def parser_pdf_file(pdf_file_path):
    read_pdf = open(pdf_file_path, 'rb')  # 打开PDF文件。
    parser_pdf = PDFParser(read_pdf)  # 用文件对象创建一个PDF文档分析器。
    pdf_document = PDFDocument(parser_pdf)  # 创建一个PDF文档。
 
    if not pdf_document.is_extractable:  # 检测文档是否提供txt转换，不提供就忽略。
        raise PDFTextExtractionNotAllowed
    else: 
        pdf_manager = PDFResourceManager()  # 创建PDF资源管理器 来管理共享资源。
        pdf_laparams = LAParams()  # 创建一个PDF参数分析器。
        pdf_device = PDFPageAggregator(pdf_manager, laparams=pdf_laparams)  # 创建一个聚合器
        pdf_interpreter = PDFPageInterpreter(pdf_manager, pdf_device)  # 创建一个PDF页面解释器对象
        # 循环遍历列表，每次处理一页的内容，pdf_document.get_pages()获取page列表
        for each_page in pdf_document.get_pages():
            pdf_interpreter.process_page(each_page)  # 使用页面解释器来读取
            layout = pdf_device.get_result()  # 这里layout是一个LTPage对象 里面存放着这个page解析出的各种对象 一般包括LTTexBox,LTFigure,LTImage,
            # LTTexBoxHorizontal等等 想要获取文本就获得对象的text属性。
            # print(layout)
            for each_info in layout:
                if isinstance(each_info, LTTextBoxHorizontal):
                    results = each_info.get_text()
                    print(results)


In [106]:
from pdfminer.pdfdevice import PDFDevice
from pdfminer.pdfpage import PDFPage
import re
import pdfplumber
from pdfminer.converter import PDFPageAggregator
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFTextExtractionNotAllowed
from pdfminer.layout import LTTextBoxHorizontal, LAParams,LTTextBoxHorizontal
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter

def get_time(file):
    fp = open(file, 'rb')
    # Create a PDF parser object associated with the file object.
    parser = PDFParser(fp)
    # Create a PDF document object that stores the document structure.
    # Supply the password for initialization.
    document = PDFDocument(parser)
    # Check if the document allows text extraction. If not, abort.
    if not document.is_extractable:
        raise PDFTextExtractionNotAllowed
    # Create a PDF resource manager object that stores shared resources.
    rsrcmgr = PDFResourceManager()
    # Create a PDF device object.
    device = PDFDevice(rsrcmgr)
    # Create a PDF interpreter object.
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    # Process each page contained in the document.
    for page in PDFPage.create_pages(document):
        interpreter.process_page(page)

    from pdfminer.layout import LAParams
    from pdfminer.converter import PDFPageAggregator

    res1 = re.compile('上周（.*?）')
    # Set parameters for analysis.
    laparams = LAParams()
    # Create a PDF page aggregator object.
    device = PDFPageAggregator(rsrcmgr, laparams=laparams)
    interpreter = PDFPageInterpreter(rsrcmgr, device)

    for page in PDFPage.create_pages(document):
        interpreter.process_page(page)
        # receive the LTPage object for the page.
        layout = device.get_result()
        for each_info in layout:
            if isinstance(each_info, LTTextBoxHorizontal):
                results = each_info.get_text()
                print(results)
                res = res1.search(results)
                if res:
                    time = res.group()
                    print(time)
                    return time
                else:
                    return ''
            

In [125]:
res1 = re.compile('上周（.*?）')
def get_time1(file):
    with pdfplumber.open(file) as pdf:
        first_page = pdf.pages[0]
        results = first_page.extract_text()[:300]
        res = res1.search(results)
        if res:
            time = res.group()[3:-1]
            print(time)
            return time
        else:
            return ''
    

In [126]:
get_time1("20210620.pdf")

2021.06.14~2021.06.18


'2021.06.14~2021.06.18'

In [129]:
table_dict = {}
def get_stocks(file,time):
    with pdfplumber.open(file) as pdf:
        page = pdf.pages[3]
        table = page.extract_tables()
    temp_array = []
    stocks = []
    res2 = re.compile('\d{6}\.\w{2}')
    for i in table[0][1:]:
        res = res2.search(str(i))
        if res:
            stocks.append(res.group())
    table_dict[time] = stocks
    print(stocks)



In [130]:
for i in pdfs:
    time = get_time1(i)
    get_stocks(i,time)
print(table_dict)


2020.06.15~2020.06.19
['000012.SZ', '000878.SZ', '600111.SH', '000962.SZ', '600997.SH', '300405.SZ', '300082.SZ', '603026.SH', '300498.SZ', '002124.SZ']
2020.08.17~2020.08.21
['002623.SZ', '600497.SH', '000831.SZ', '600366.SH', '603799.SH', '601969.SH', '000898.SZ', '601678.SH', '002455.SZ']
2020.10.12~2020.10.16
['002428.SZ', '601699.SH', '600803.SH', '002221.SZ', '600586.SH', '000635.SZ', '002068.SZ', '601678.SH', '000949.SZ', '601118.SH']
2021.05.17~2021.05.21
['600876.SH', '601958.SH', '603993.SH', '000962.SZ', '600740.SH', '600803.SH', '000545.SZ', '002064.SZ', '002895.sz', '002258.SZ']
2020.12.07~2020.12.11
['002613.SZ', '002182.SZ', '601666.SH', '000159.SZ', '000683.SZ', '002221.SZ', '002386.SZ', '002068.SZ', '002258.SZ', '002234.SZ']
2021.04.26~2021.05.07
['000878.SZ', '002182.SZ', '600508.SH', '000778.SZ', '002136.SZ', '000703.SZ', '002588.SZ', '002258.SZ', '600596.SH', '600737.SH']
2020.09.28~2020.10.09
['600585.SH', '000012.SZ', '002428.SZ', '600348.SH', '002274.SZ', '000822

In [133]:
import pandas as pd
table = pd.Series(table_dict)
table

2020.06.15~2020.06.19    [000012.SZ, 000878.SZ, 600111.SH, 000962.SZ, 6...
2020.08.17~2020.08.21    [002623.SZ, 600497.SH, 000831.SZ, 600366.SH, 6...
2020.10.12~2020.10.16    [002428.SZ, 601699.SH, 600803.SH, 002221.SZ, 6...
2021.05.17~2021.05.21    [600876.SH, 601958.SH, 603993.SH, 000962.SZ, 6...
2020.12.07~2020.12.11    [002613.SZ, 002182.SZ, 601666.SH, 000159.SZ, 0...
2021.04.26~2021.05.07    [000878.SZ, 002182.SZ, 600508.SH, 000778.SZ, 0...
2020.09.28~2020.10.09    [600585.SH, 000012.SZ, 002428.SZ, 600348.SH, 0...
2020.08.10~2020.08.14    [000012.SZ, 600497.SH, 600392.SH, 000795.SZ, 6...
2020.07.06~2020.07.10    [000012.SZ, 000630.SZ, 601600.SH, 600531.SH, 0...
2020.03.08~2020.03.12    [002182.SZ, 601216.SH, 002601.SZ, 601233.SH, 0...
2020.11.09~2020.11.13    [601958.SH, 603993.SH, 002110.SZ, 000635.SZ, 0...
2020.09.14~2020.09.18    [600585.SH, 000012.SZ, 601088.SH, 000937.SZ, 0...
2020.12.14~2020.12.18    [002623.SZ, 002171.SZ, 300127.SZ, 600508.SH, 6...
2020.12.28~2020.12.31    

In [134]:
table.to_csv('result.csv')